In [0]:
import matplotlib.pyplot as plt
import PIL
import tensorflow as tf
import numpy as np
import os

from tensorflow.python.keras.models import Model, Sequential
from tensorflow.python.keras.layers import Dense, Flatten, Dropout
from tensorflow.python.keras.applications.vgg19 import VGG19
from tensorflow.python.keras.applications.inception_v3 import preprocess_input, decode_predictions
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.optimizers import Adam, RMSprop

train_dir = "/content/drive/My Drive/ADNI_C/test/des/ADtrain"
test_dir = "/content/drive/My Drive/ADNI_C/test/des/ADtest"

model = VGG19(include_top=False,input_shape=(224, 224, 3), weights='imagenet')
input_shape = (224,224)

datagen_train = ImageDataGenerator(rescale=1./255)
datagen_test = ImageDataGenerator(rescale=1./255)


batch_size = 20

generator_train = datagen_train.flow_from_directory(directory=train_dir,
                                                    target_size=input_shape,
                                                    batch_size=batch_size,
                                                    shuffle=True)

generator_test = datagen_test.flow_from_directory(directory=test_dir,
                                                  target_size=input_shape,
                                                  batch_size=batch_size,
                                                  shuffle=False)

steps_test = generator_test.n / batch_size


def path_join(dirname, filenames):
    return [os.path.join(dirname, filename) for filename in filenames]

image_paths_train = path_join(train_dir, generator_train.filenames)
image_paths_test = path_join(test_dir, generator_test.filenames)

cls_train = generator_train.classes
cls_test = generator_test.classes


class_names = list(generator_train.class_indices.keys())
num_classes = generator_train.num_classes

transfer_layer = model.get_layer('block5_pool')
conv_model = Model(inputs=model.input, outputs=transfer_layer.output)

#for layer in conv_model.layers:
#    layer.trainable = False
    
# Start a new Keras Sequential model.
new_model = Sequential()

# Add the convolutional part of the VGG16 model from above.
new_model.add(conv_model)

# Flatten the output of the VGG16 model because it is from a
# convolutional layer.
new_model.add(Flatten())

# Add a dense (aka. fully-connected) layer.
# This is for combining features that the VGG16 model has
# recognized in the image.
new_model.add(Dropout(0.5))

new_model.add(Dense(1024, activation='relu'))

# Add a dropout-layer which may prevent overfitting and
# improve generalization ability to unseen data e.g. the test-set.
#new_model.add(Dense(512, activation='relu'))


# Add the final layer for the actual classification.
new_model.add(Dense(num_classes, activation='softmax'))

optimizer = Adam(lr=1e-5)
loss = 'categorical_crossentropy'
metrics = ['categorical_accuracy']


def print_layer_trainable():
    for layer in conv_model.layers:
        print("{0}:\t{1}".format(layer.trainable, layer.name))


print_layer_trainable()

new_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

epochs = 5
steps_per_epoch = 100


history = new_model.fit_generator(generator=generator_train,
                                  epochs=epochs,
                                  steps_per_epoch=steps_per_epoch,
                                  validation_data=generator_test,
                                  validation_steps=steps_test)
 

W0803 14:18:56.023811 140501654341504 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


80142336/80134624 [==============================] - 2s 0us/step
Found 11192 images belonging to 3 classes.
Found 2798 images belonging to 3 classes.
True:	input_1
True:	block1_conv1
True:	block1_conv2
True:	block1_pool
True:	block2_conv1
True:	block2_conv2
True:	block2_pool
True:	block3_conv1
True:	block3_conv2
True:	block3_conv3
True:	block3_conv4
True:	block3_pool
True:	block4_conv1
True:	block4_conv2
True:	block4_conv3
True:	block4_conv4
True:	block4_pool
True:	block5_conv1
True:	block5_conv2
True:	block5_conv3
True:	block5_conv4
True:	block5_pool
Epoch 1/5
100/100 [==============================] - 1257s 13s/step - loss: 1.1666 - categorical_accuracy: 0.4021 - val_loss: 1.1220 - val_categorical_accuracy: 0.4503
Epoch 2/5
100/100 [==============================] - 475s 5s/step - loss: 1.0561 - categorical_accuracy: 0.4545 - val_loss: 0.9240 - val_categorical_accuracy: 0.5490
Epoch 3/5
100/100 [==============================] - 517s 5s/step - loss: 0.9514 - categorical_accuracy: 0.5

In [0]:
epochs = 5
steps_per_epoch = 100


history = new_model.fit_generator(generator=generator_train,
                                  epochs=epochs,
                                  steps_per_epoch=steps_per_epoch,
                                  validation_data=generator_test,
                                  validation_steps=steps_test)

Epoch 1/5
100/100 [==============================] - 78s 781ms/step - loss: 0.5092 - categorical_accuracy: 0.7665 - val_loss: 0.4288 - val_categorical_accuracy: 0.8081
Epoch 2/5
100/100 [==============================] - 84s 843ms/step - loss: 0.4342 - categorical_accuracy: 0.8070 - val_loss: 0.3524 - val_categorical_accuracy: 0.8499
Epoch 3/5
100/100 [==============================] - 108s 1s/step - loss: 0.3990 - categorical_accuracy: 0.8305 - val_loss: 0.3641 - val_categorical_accuracy: 0.8352
Epoch 4/5
100/100 [==============================] - 81s 811ms/step - loss: 0.3528 - categorical_accuracy: 0.8514 - val_loss: 0.2888 - val_categorical_accuracy: 0.8749
Epoch 5/5
100/100 [==============================] - 96s 956ms/step - loss: 0.3380 - categorical_accuracy: 0.8490 - val_loss: 0.2304 - val_categorical_accuracy: 0.8981


In [0]:
epochs = 5
steps_per_epoch = 100


history = new_model.fit_generator(generator=generator_train,
                                  epochs=epochs,
                                  steps_per_epoch=steps_per_epoch,
                                  validation_data=generator_test,
                                  validation_steps=steps_test)

Epoch 1/5
100/100 [==============================] - 64s 636ms/step - loss: 0.2578 - categorical_accuracy: 0.8870 - val_loss: 0.2092 - val_categorical_accuracy: 0.9114
Epoch 2/5
100/100 [==============================] - 63s 632ms/step - loss: 0.2580 - categorical_accuracy: 0.8880 - val_loss: 0.1966 - val_categorical_accuracy: 0.9174
Epoch 3/5
100/100 [==============================] - 62s 623ms/step - loss: 0.2497 - categorical_accuracy: 0.8986 - val_loss: 0.1869 - val_categorical_accuracy: 0.9192
Epoch 4/5
100/100 [==============================] - 62s 625ms/step - loss: 0.2213 - categorical_accuracy: 0.9075 - val_loss: 0.1858 - val_categorical_accuracy: 0.9196
Epoch 5/5
100/100 [==============================] - 63s 627ms/step - loss: 0.1979 - categorical_accuracy: 0.9135 - val_loss: 0.1677 - val_categorical_accuracy: 0.9317


In [0]:
epochs = 5
steps_per_epoch = 100


history = new_model.fit_generator(generator=generator_train,
                                  epochs=epochs,
                                  steps_per_epoch=steps_per_epoch,
                                  validation_data=generator_test,
                                  validation_steps=steps_test)

Epoch 1/5
100/100 [==============================] - 61s 605ms/step - loss: 0.1960 - categorical_accuracy: 0.9195 - val_loss: 0.1515 - val_categorical_accuracy: 0.9346
Epoch 2/5
100/100 [==============================] - 63s 628ms/step - loss: 0.1767 - categorical_accuracy: 0.9255 - val_loss: 0.1430 - val_categorical_accuracy: 0.9400
Epoch 3/5
100/100 [==============================] - 63s 631ms/step - loss: 0.1868 - categorical_accuracy: 0.9242 - val_loss: 0.1315 - val_categorical_accuracy: 0.9510
Epoch 4/5
100/100 [==============================] - 63s 625ms/step - loss: 0.1379 - categorical_accuracy: 0.9420 - val_loss: 0.1217 - val_categorical_accuracy: 0.9482
Epoch 5/5
100/100 [==============================] - 63s 625ms/step - loss: 0.1503 - categorical_accuracy: 0.9415 - val_loss: 0.1124 - val_categorical_accuracy: 0.9582


In [0]:
import matplotlib.pyplot as plt
import PIL
import tensorflow as tf
import numpy as np
import os

from tensorflow.python.keras.models import Model, Sequential
from tensorflow.python.keras.layers import Dense, Flatten, Dropout
from tensorflow.python.keras.applications.inception_v3 import InceptionV3
from tensorflow.python.keras.applications.inception_v3 import preprocess_input, decode_predictions
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.optimizers import Adam, RMSprop

train_dir = "/content/drive/My Drive/ADNI_C/test/des/ADtrain"
test_dir = "/content/drive/My Drive/ADNI_C/test/des/ADtest"

model = InceptionV3(include_top=False,input_shape=(224, 224, 3), weights='imagenet')
input_shape = (224,224)

datagen_train = ImageDataGenerator(rescale=1./255)
datagen_test = ImageDataGenerator(rescale=1./255)


batch_size = 20

generator_train = datagen_train.flow_from_directory(directory=train_dir,
                                                    target_size=input_shape,
                                                    batch_size=batch_size,
                                                    shuffle=True)

generator_test = datagen_test.flow_from_directory(directory=test_dir,
                                                  target_size=input_shape,
                                                  batch_size=batch_size,
                                                  shuffle=False)

steps_test = generator_test.n / batch_size


def path_join(dirname, filenames):
    return [os.path.join(dirname, filename) for filename in filenames]

image_paths_train = path_join(train_dir, generator_train.filenames)
image_paths_test = path_join(test_dir, generator_test.filenames)

cls_train = generator_train.classes
cls_test = generator_test.classes


class_names = list(generator_train.class_indices.keys())
num_classes = generator_train.num_classes

transfer_layer = model.get_layer('mixed10')
conv_model = Model(inputs=model.input, outputs=transfer_layer.output)

#for layer in conv_model.layers:
#    layer.trainable = False
    
# Start a new Keras Sequential model.
new_model = Sequential()

# Add the convolutional part of the VGG16 model from above.
new_model.add(conv_model)

# Flatten the output of the VGG16 model because it is from a
# convolutional layer.
new_model.add(Flatten())

# Add a dense (aka. fully-connected) layer.
# This is for combining features that the VGG16 model has
# recognized in the image.
new_model.add(Dropout(0.5))

new_model.add(Dense(1024, activation='relu'))

# Add a dropout-layer which may prevent overfitting and
# improve generalization ability to unseen data e.g. the test-set.
#new_model.add(Dense(512, activation='relu'))


# Add the final layer for the actual classification.
new_model.add(Dense(num_classes, activation='softmax'))

optimizer = Adam(lr=1e-5)
loss = 'categorical_crossentropy'
metrics = ['categorical_accuracy']


def print_layer_trainable():
    for layer in conv_model.layers:
        print("{0}:\t{1}".format(layer.trainable, layer.name))


print_layer_trainable()

new_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

epochs = 5
steps_per_epoch = 100


history = new_model.fit_generator(generator=generator_train,
                                  epochs=epochs,
                                  steps_per_epoch=steps_per_epoch,
                                  validation_data=generator_test,
                                  validation_steps=steps_test)
 

W0805 12:33:04.972873 140059230308224 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Found 11192 images belonging to 3 classes.
Found 2798 images belonging to 3 classes.
True:	input_1
True:	conv2d
True:	batch_normalization
True:	activation
True:	conv2d_1
True:	batch_normalization_1
True:	activation_1
True:	conv2d_2
True:	batch_normalization_2
True:	activation_2
True:	max_pooling2d
True:	conv2d_3
True:	batch_normalization_3
True:	activation_3
True:	conv2d_4
True:	batch_normalization_4
True:	activation_4
True:	max_pooling2d_1
True:	conv2d_8
True:	batch_normalization_8
True:	activation_8
True:	conv2d_6
True:	conv2d_9
True:	batch_normalization_6
True:	batch_normalization_9
True:	activation_6
True:	activation_9
True:	average_pooling2d
True:	conv2d_5
True:	conv2d_7
True:	conv2d_10
True:	conv2d_11
True:	batch_normalization_5
True:	batch_normalization_7
True:	batch_normalization_10
True:	batch_normalization_11
True:	activation_5
True:	activation_7
True:	activation_10
True:	activation_11
True:	mixed0
True:	conv2d_15
True:	batch_normalization_15
True:	activation_15
True:	conv2d_

In [2]:
epochs = 5
steps_per_epoch = 100


history = new_model.fit_generator(generator=generator_train,
                                  epochs=epochs,
                                  steps_per_epoch=steps_per_epoch,
                                  validation_data=generator_test,
                                  validation_steps=steps_test)

Epoch 1/5
100/100 [==============================] - 134s 1s/step - loss: 0.5398 - categorical_accuracy: 0.7800 - val_loss: 0.6591 - val_categorical_accuracy: 0.7094
Epoch 2/5
100/100 [==============================] - 89s 892ms/step - loss: 0.4910 - categorical_accuracy: 0.8067 - val_loss: 0.5517 - val_categorical_accuracy: 0.7609
Epoch 3/5
100/100 [==============================] - 97s 968ms/step - loss: 0.4553 - categorical_accuracy: 0.8115 - val_loss: 0.4825 - val_categorical_accuracy: 0.7963
Epoch 4/5
100/100 [==============================] - 118s 1s/step - loss: 0.4508 - categorical_accuracy: 0.8165 - val_loss: 0.4512 - val_categorical_accuracy: 0.8059
Epoch 5/5
100/100 [==============================] - 67s 670ms/step - loss: 0.4184 - categorical_accuracy: 0.8225 - val_loss: 0.3902 - val_categorical_accuracy: 0.8377


In [3]:
epochs = 5
steps_per_epoch = 100


history = new_model.fit_generator(generator=generator_train,
                                  epochs=epochs,
                                  steps_per_epoch=steps_per_epoch,
                                  validation_data=generator_test,
                                  validation_steps=steps_test)

Epoch 1/5
100/100 [==============================] - 41s 413ms/step - loss: 0.1886 - categorical_accuracy: 0.9390 - val_loss: 0.3564 - val_categorical_accuracy: 0.8595
Epoch 2/5
100/100 [==============================] - 47s 472ms/step - loss: 0.1922 - categorical_accuracy: 0.9335 - val_loss: 0.3382 - val_categorical_accuracy: 0.8628
Epoch 3/5
100/100 [==============================] - 35s 354ms/step - loss: 0.1998 - categorical_accuracy: 0.9285 - val_loss: 0.3440 - val_categorical_accuracy: 0.8628
Epoch 4/5
100/100 [==============================] - 35s 352ms/step - loss: 0.1713 - categorical_accuracy: 0.9370 - val_loss: 0.3711 - val_categorical_accuracy: 0.8556
Epoch 5/5
100/100 [==============================] - 35s 351ms/step - loss: 0.1760 - categorical_accuracy: 0.9370 - val_loss: 0.3178 - val_categorical_accuracy: 0.8695


In [4]:
epochs = 5
steps_per_epoch = 100


history = new_model.fit_generator(generator=generator_train,
                                  epochs=epochs,
                                  steps_per_epoch=steps_per_epoch,
                                  validation_data=generator_test,
                                  validation_steps=steps_test)

Epoch 1/5
100/100 [==============================] - 35s 351ms/step - loss: 0.0651 - categorical_accuracy: 0.9839 - val_loss: 0.3242 - val_categorical_accuracy: 0.8753
Epoch 2/5
100/100 [==============================] - 35s 351ms/step - loss: 0.0676 - categorical_accuracy: 0.9820 - val_loss: 0.3297 - val_categorical_accuracy: 0.8710
Epoch 3/5
100/100 [==============================] - 35s 349ms/step - loss: 0.0767 - categorical_accuracy: 0.9770 - val_loss: 0.3465 - val_categorical_accuracy: 0.8731
Epoch 4/5
100/100 [==============================] - 35s 352ms/step - loss: 0.0714 - categorical_accuracy: 0.9750 - val_loss: 0.3288 - val_categorical_accuracy: 0.8767
Epoch 5/5
100/100 [==============================] - 35s 351ms/step - loss: 0.0794 - categorical_accuracy: 0.9715 - val_loss: 0.2905 - val_categorical_accuracy: 0.8985
